
## Scoring algorithm

##### Create continuous same size bins based on individual factor distribution (custom categories)

- assuming dataframe where every row is indexed by neighborhood
- columns: pop_density (1-5), rent (1-5), competition(1-5), connectivity (1-5), avg. price (1-4), avg. rating (0-5)get user inputs and weights
- add a factors of no. of reviews/no. of checkins?

In [2]:
import pandas as pd

In [41]:
u_n = 10 #user enters no. of results he wants to see
category = 'Chinese Restaurant' #user enters category
scores_matrix = pd.read_csv('neighbor_category_scores_matrix.csv')

res = scores_matrix[category].values.argsort()[::-1][:100]
scores = pd.DataFrame(scores_matrix.ix[res, category])
neighbors = res
unsup_scores = scores[category].values
print(neighbors)

[86 89 87 43 83  8  2  9  5 69 73 76 95 25 56  0 18  4 17 31 13 77 93 97 72
 55 47 30 92 42 60 98 64 49 20 88 41 79 59  3 40 96 62 21 84 37 32 71 63 48
 78 81 74 61 28 53  1 50 45 85 94 15 38 44 22 29 51 33 26  6 82 58  7 24 14
 27 91 54 75 67 99 68 19 90 57 70 52 46 39 36 35 23 10 11 12 34 66 80 16 65]


### For all categories other than Chinese food

In [44]:
locations = pd.read_csv("centroids_of_neighborhoods.csv")

recommended_neighbors = neighbors[:u_n]
recommended_locations = locations.ix[recommended_neighbors,]
print(recommended_locations)

    Unnamed: 0        lat        lng
86          86  40.680512 -73.873323
89          89  40.714038 -73.744252
87          87  40.883641 -73.868689
43          43  40.701749 -73.920029
83          83  40.757970 -73.831294
8            8  40.759391 -74.217670
2            2  40.672290 -73.731013
9            9  40.719204 -73.992479
5            5  40.746493 -73.890381
69          69  40.739374 -74.062017


### For Chinese food

In [31]:

alpha = 1

def pop_score(df, n,u_pop):
    return np.exp(alpha * abs(df.ix[n,'population_density'],u_pop))

def rent_score(df, n, u_rent):
    
    if df.ix[n,'rent'] <= u_rent:
        return 1
    
    else:
        return np.exp(alpha * abs(df.ix[n,'rent'],u_rent))
    
def comp_score(df, n, u_comp):
    return np.exp(alpha * abs(df.ix[n,'competition'],u_comp))

def conn_score(df, n, u_conn):
    
    if df[n]['connectivity'] >= u_conn:
        return 1
    
    else:
        return np.exp(alpha * abs(df.ix[n,'connectivity'],u_conn))

#assuming user wants to open up in a location where he is not competing on price?
def price_score(df, n, u_price):
    return np.exp(alpha * abs(df.ix[n,'avg_price'],u_price))


In [ ]:
# all score values should be between 0 and 1
# all weights should be between 0 and 1


def score(df, n, weights, u_pop, u_rent, u_comp, u_conn, u_price, unsup_score):
    
    a = pop_score(df,n,u_pop)
    b = rent_score(df, n, u_rent)
    c = comp_score(df, n, u_comp)
    d = conn_score(df, n, u_conn)
    e = price_score(df, n, u_price)
    
    final_score = (weights[0] * unsup_score[n]) + (weights[1] * a)+ (weights[2] * b) + (weights[3] * c) + 
    (weights[4] * d) + (weights[5] * e) + (weights[6] * df.ix[n,'rating']/5) 
    
    return final_score

In [80]:
# evaluation - no. of existing such restaurants , no. of resturants that closed down
# for recommended locations, just pull from big dataframe

#take weights, u_pop, u_rent, u_comp, u_conn, u_price from user
prefscores = np.zeros((100,))

for n in df.index.values:
    prefscores[n] = score(df, n, weights, u_pop, u_rent, u_comp, u_conn, u_price, unsup_score)
    
recommended_neighbors = prefscores.argsort()[::-1][:u_n]
recommended_scores = prefscores[recommended_neighbors]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [36]:
recommended_locations = locations.ix[recommended_neighbors,]